# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports


In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from statistics import mean
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token


In [2]:
stocks = pd.read_csv(r"C:\Users\conta\OneDrive\Desktop\Project Pro\algorithmic-trading-python-master\algorithmic-trading-python-master\starter_files\sp_500_stocks.csv")
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call


In [3]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 80555952, 'calculationPrice': 'tops', 'change': -0.73, 'changePercent': -0.00485, 'close': 0, 'closeSource': 'fflioica', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 157.41, 'iexAskSize': 408, 'iexBidPrice': 161.98, 'iexBidSize': 102, 'iexClose': 160.42, 'iexCloseTime': 1679074376465, 'iexLastUpdated': 1714536846214, 'iexMarketPercent': 0.0171301285761223, 'iexOpen': 158.247, 'iexOpenTime': 1703845492465, 'iexRealtimePrice': 156.71, 'iexRealtimeSize': 305, 'iexVolume': 218049, 'lastTradeTime': 1713830333087, 'latestPrice': 156.96, 'latestSource': 'IEX real time price', 'latestTime': '10:11:05 AM', 'latestUpdate': 1738803381883, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2584896767824, 'oddLotDela

## Parsing Our API Call


In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

26.51

## Executing A Batch API Call & Building Our DataFrame



In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW': # there are some errors with these stocks
            continue
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,137.030,31.57,N/A
1,AAL,14.915,-5.12,N/A
2,AAP,178.620,20.24,N/A
3,AAPL,160.030,26.52,N/A
4,ABBV,143.500,20.18,N/A
...,...,...,...,...
496,YUM,118.067,23.93,N/A
497,ZBH,118.400,109.8,N/A
498,ZBRA,303.900,33.15,N/A
499,ZION,59.770,6.84,N/A


## Removing Glamour Stocks



In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,44.460,3.27,N/A
1,AIG,55.360,3.74,N/A
2,NUE,123.460,3.86,N/A
3,PVH,58.470,4.35,N/A
4,PHM,40.340,4.52,N/A
5,COF,104.690,4.59,N/A
6,DHI,71.590,4.66,N/A
7,HPQ,28.290,4.74,N/A
8,HPE,13.660,4.83,N/A
9,PFG,78.500,4.93,N/A


## Calculating the Number of Shares to Buy


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,44.460,3.27,449
1,AIG,55.360,3.74,361
2,NUE,123.460,3.86,161
3,PVH,58.470,4.35,342
4,PHM,40.340,4.52,495
5,COF,104.690,4.59,191
6,DHI,71.590,4.66,279
7,HPQ,28.290,4.74,706
8,HPE,13.660,4.83,1464
9,PFG,78.500,4.93,254


## Building a Better (and More Realistic) Value Strategy


In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW': # there are some errors with these stocks
            continue
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.680,N/A,31.46,N/A,8.11,N/A,6.05,N/A,21.923280,N/A,11.171731,N/A,N/A
1,AAL,15.071,N/A,-5.18,N/A,-1.13,N/A,0.2284,N/A,111.254929,N/A,0.830664,N/A,N/A
2,AAP,170.230,N/A,19.91,N/A,3.82,N/A,0.9495,N/A,10.931901,N/A,2.328084,N/A,N/A
3,AAPL,161.660,N/A,26.09,N/A,43.92,N/A,6.61,N/A,19.854534,N/A,15.084488,N/A,N/A
4,ABBV,143.000,N/A,20.26,N/A,17.48,N/A,4.4,N/A,10.836028,N/A,7.589425,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.320,N/A,23.4,N/A,-3.98,N/A,5.04,N/A,19.614588,N/A,8.621543,N/A,N/A
497,ZBH,122.700,N/A,109.7,N/A,2.1,N/A,3.4,N/A,18.615735,N/A,5.915324,N/A,N/A
498,ZBRA,304.500,N/A,33.18,N/A,6.3,N/A,2.66,N/A,16.545500,N/A,6.750331,N/A,N/A
499,ZION,59.910,N/A,6.82,N/A,1.76,N/A,2.53,N/A,5.118198,N/A,2.446971,N/A,N/A


## Dealing With Missing Data in Our DataFrame


In [13]:
rv_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   501 non-null    object 
 1   Price                    501 non-null    float64
 2   Number of Shares to Buy  501 non-null    object 
 3   Price-to-Earnings Ratio  491 non-null    object 
 4   PE Percentile            501 non-null    object 
 5   Price-to-Book Ratio      481 non-null    object 
 6   PB Percentile            501 non-null    object 
 7   Price-to-Sales Ratio     481 non-null    object 
 8   PS Percentile            501 non-null    object 
 9   EV/EBITDA                481 non-null    float64
 10  EV/EBITDA Percentile     501 non-null    object 
 11  EV/GP                    481 non-null    float64
 12  EV/GP Percentile         501 non-null    object 
 13  RV Score                 501 non-null    object 
dtypes: float64(3), object(11)


In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,185.900,N/A,59.58,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,291.960,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,283.200,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,97.180,N/A,48.35,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.92,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.130,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,51.140,N/A,14.54,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.400,N/A,32.68,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,32.040,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,105.080,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [15]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [17]:
rv_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   501 non-null    object 
 1   Price                    501 non-null    float64
 2   Number of Shares to Buy  501 non-null    object 
 3   Price-to-Earnings Ratio  501 non-null    float64
 4   PE Percentile            501 non-null    object 
 5   Price-to-Book Ratio      501 non-null    float64
 6   PB Percentile            501 non-null    object 
 7   Price-to-Sales Ratio     501 non-null    float64
 8   PS Percentile            501 non-null    object 
 9   EV/EBITDA                501 non-null    float64
 10  EV/EBITDA Percentile     501 non-null    object 
 11  EV/GP                    501 non-null    float64
 12  EV/GP Percentile         501 non-null    object 
 13  RV Score                 501 non-null    object 
dtypes: float64(6), object(8)
m

## Calculating Value Percentiles



In [18]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.680,N/A,31.46,0.740519,8.11,0.784431,6.0500,0.836327,21.923280,0.822355,11.171731,0.810379,N/A
1,AAL,15.071,N/A,-5.18,0.0339321,-1.13,0.0598802,0.2284,0.011976,111.254929,0.996008,0.830664,0.0339321,N/A
2,AAP,170.230,N/A,19.91,0.473054,3.82,0.556886,0.9495,0.143713,10.931901,0.355289,2.328084,0.139721,N/A
3,AAPL,161.660,N/A,26.09,0.644711,43.92,0.978044,6.6100,0.852295,19.854534,0.762475,15.084488,0.922156,N/A
4,ABBV,143.000,N/A,20.26,0.48503,17.48,0.934132,4.4000,0.730539,10.836028,0.345309,7.589425,0.61477,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.320,N/A,23.40,0.552894,-3.98,0.0558882,5.0400,0.791417,19.614588,0.754491,8.621543,0.682635,N/A
497,ZBH,122.700,N/A,109.70,0.982036,2.10,0.330339,3.4000,0.582834,18.615735,0.740519,5.915324,0.43513,N/A
498,ZBRA,304.500,N/A,33.18,0.764471,6.30,0.730539,2.6600,0.47505,16.545500,0.666667,6.750331,0.50499,N/A
499,ZION,59.910,N/A,6.82,0.115768,1.76,0.248503,2.5300,0.451098,5.118198,0.0938124,2.446971,0.147705,N/A


## Calculating the RV Score


In [19]:

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.680,N/A,31.46,0.740519,8.11,0.784431,6.0500,0.836327,21.923280,0.822355,11.171731,0.810379,0.798802
1,AAL,15.071,N/A,-5.18,0.0339321,-1.13,0.0598802,0.2284,0.011976,111.254929,0.996008,0.830664,0.0339321,0.227146
2,AAP,170.230,N/A,19.91,0.473054,3.82,0.556886,0.9495,0.143713,10.931901,0.355289,2.328084,0.139721,0.333733
3,AAPL,161.660,N/A,26.09,0.644711,43.92,0.978044,6.6100,0.852295,19.854534,0.762475,15.084488,0.922156,0.831936
4,ABBV,143.000,N/A,20.26,0.48503,17.48,0.934132,4.4000,0.730539,10.836028,0.345309,7.589425,0.61477,0.621956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,120.320,N/A,23.40,0.552894,-3.98,0.0558882,5.0400,0.791417,19.614588,0.754491,8.621543,0.682635,0.567465
497,ZBH,122.700,N/A,109.70,0.982036,2.10,0.330339,3.4000,0.582834,18.615735,0.740519,5.915324,0.43513,0.614172
498,ZBRA,304.500,N/A,33.18,0.764471,6.30,0.730539,2.6600,0.47505,16.545500,0.666667,6.750331,0.50499,0.628343
499,ZION,59.910,N/A,6.82,0.115768,1.76,0.248503,2.5300,0.451098,5.118198,0.0938124,2.446971,0.147705,0.211377


## Selecting the 50 Best Value Stocks¶



In [20]:
rv_dataframe.sort_values('RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,30.270,N/A,5.80,0.0838323,0.7594,0.0758483,0.1800,0.00598802,4.077625,0.0618762,1.062140,0.0518962,0.0558882
1,AIG,54.740,N/A,3.62,0.0499002,0.9573,0.0978044,0.7220,0.10978,2.125112,0.0159681,0.696001,0.0159681,0.0578842
2,PVH,57.570,N/A,4.43,0.0538922,0.7120,0.0718563,0.4026,0.0399202,4.621262,0.0858283,0.989931,0.0499002,0.0602794
3,UNM,41.030,N/A,7.73,0.131737,0.8405,0.0838323,0.6794,0.0998004,4.084413,0.0638723,0.663917,0.00998004,0.0778443
4,SYF,33.449,N/A,4.83,0.0658683,1.2400,0.140719,0.8721,0.135729,2.896829,0.0219561,0.975893,0.0479042,0.0824351
5,GM,43.751,N/A,8.33,0.137725,0.9565,0.0958084,0.4588,0.0538922,2.530753,0.0179641,1.978923,0.111776,0.0834331
6,CAH,68.490,N/A,-20.24,0.0179641,-26.3800,0.0259481,0.1035,0.00199601,7.092985,0.181637,2.836956,0.191617,0.0838323
7,COF,106.340,N/A,4.54,0.0578842,0.7366,0.0738523,1.1700,0.197605,3.059948,0.0259481,1.194496,0.0678643,0.0846307
8,HPQ,27.956,N/A,4.70,0.0618762,-11.9700,0.0439122,0.4310,0.0479042,5.947324,0.125749,2.621140,0.167665,0.0894212
9,PHM,40.537,N/A,4.49,0.0558882,1.1700,0.128743,0.6033,0.0818363,3.524168,0.0379242,2.418740,0.145709,0.09002


## Calculating the Number of Shares to Buy


In [21]:
portfolio_input()

Enter the value of your portfolio:1000000


In [22]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\conta\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,30.270,660,5.80,0.0838323,0.7594,0.0758483,0.1800,0.00598802,4.077625,0.0618762,1.062140,0.0518962,0.0558882
1,AIG,54.740,365,3.62,0.0499002,0.9573,0.0978044,0.7220,0.10978,2.125112,0.0159681,0.696001,0.0159681,0.0578842
2,PVH,57.570,347,4.43,0.0538922,0.7120,0.0718563,0.4026,0.0399202,4.621262,0.0858283,0.989931,0.0499002,0.0602794
3,UNM,41.030,487,7.73,0.131737,0.8405,0.0838323,0.6794,0.0998004,4.084413,0.0638723,0.663917,0.00998004,0.0778443
4,SYF,33.449,597,4.83,0.0658683,1.2400,0.140719,0.8721,0.135729,2.896829,0.0219561,0.975893,0.0479042,0.0824351
5,GM,43.751,457,8.33,0.137725,0.9565,0.0958084,0.4588,0.0538922,2.530753,0.0179641,1.978923,0.111776,0.0834331
6,CAH,68.490,292,-20.24,0.0179641,-26.3800,0.0259481,0.1035,0.00199601,7.092985,0.181637,2.836956,0.191617,0.0838323
7,COF,106.340,188,4.54,0.0578842,0.7366,0.0738523,1.1700,0.197605,3.059948,0.0259481,1.194496,0.0678643,0.0846307
8,HPQ,27.956,715,4.70,0.0618762,-11.9700,0.0439122,0.4310,0.0479042,5.947324,0.125749,2.621140,0.167665,0.0894212
9,PHM,40.537,493,4.49,0.0558882,1.1700,0.128743,0.6033,0.0818363,3.524168,0.0379242,2.418740,0.145709,0.09002


## Formatting Our Excel Output



In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File


In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [26]:
writer.save()